# LongSlit-01:   Classify images  

## Overview
1. Start CCD reduction using ccdproc
2. Creating lists of images 
3. Examples

<pre>
Máster en Astrofísica UCM  -- Técnicas Experimentales en Astrofísica  
Jaime Zamorano, Nicolás Cardiel and Sergio pascual

Version 1.0 2021/02/09  
</pre>

Note that the ``astropy`` package should be installed. In this sense, have a look to the
astropy installation description: https://docs.astropy.org/en/stable/install.html.

In [ ]:
from astropy.io import fits
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from ccdproc import CCDData, Combiner, subtract_overscan

In [ ]:
plt.style.use('./tea.mplstyle')    # Some parameters for nicer graphs

### Working with images in a directory
We will use the files of the first night of observations at NOT (Nordic Optical Telescope) 2008 that can be downloaded from   ftp://astrax.fis.ucm.es/pub/users/jaz/NOT_2008_04_12-14/N1/
or 
http://guaix.fis.ucm.es/~jaz/master_TEA/observaciones_NOT_2008/N1/

``directory`` should point to the directory with the downloaded images

In [ ]:
directory='/Users/jzamorano/Desktop/INT_1997/N1/'

Let\'s create a list containing all the FITS files in that directory 

In [ ]:
# only works in notebooks
# we can pass the python variable to shell
filelist = !ls $directory/*.fits

In [ ]:
from pathlib import Path
p = Path(directory)
filelist = list(sorted(p.glob('*.fits')))
print(filelist[10:14])    # printing only from 10 to 20

Let\'s open one of the files to explore the HEADER keywords

In [ ]:
HDUList_object = fits.open(filelist[-8])
HDUList_object.info()

In [ ]:
primary_HDU = HDUList_object[0]

In [ ]:
# Extract primary header
primary_header = primary_HDU.header

# Index header object with keyword name and print value
print(primary_header['OBJECT'],primary_header['EXPTIME'],primary_header['OBSTYPE'])

In [ ]:
for key in primary_header:
    print('{:8} = {}'.format(key, primary_header[key]))

Each telescope/instrument has differences in the ``keyword``names.  

## Complete list and some info

In [ ]:
for i in range(len(filelist)):
    file = filelist[i]
    HDUList_object = fits.open(file)
    primary_HDU = HDUList_object[0]
    header = primary_HDU.header
    print(i,str(file)[37:],header['EXPTIME'],'   ',header['OBJECT'],header['CCDATEMP'])

### Reading the files

In [ ]:
image_data, header , filename = [] , [], []
for i in range(len(filelist)):
    file = filelist[i]
    HDUList_object = fits.open(file)
    primary_HDU = HDUList_object[0]
    header.append(primary_HDU.header)
    image_data.append(primary_HDU.data)
    filename.append(str(file)[37:])
    HDUList_object.close()
print(filename)

### Displaying some files

#### BIAS files (at the beginning of the night)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=5, figsize=(14, 12))
n = 0
for i in range(3,8):
    img = ax[n].imshow(image_data[i], cmap='gray', origin='lower',vmin=685,vmax=700)
    ax[n].set_xlabel('X axis')
    ax[n].set_ylabel('Y axis')
    ax[n].text(100,200,filename[i],color='k',fontsize=20)
    ax[n].text(400,200,round(np.mean(image_data[i][100:200,600:800]),2),color='k',fontsize=20)
    divider = make_axes_locatable(ax[n])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(img, cax=cax)
    n = n+1

#### BIAS files (at the end of the night)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=5, figsize=(14, 12))
n = 0
for i in range(31,36):
    img = ax[n].imshow(image_data[i], cmap='gray', origin='lower',vmin=685,vmax=700)
    ax[n].set_xlabel('X axis')
    ax[n].set_ylabel('Y axis')
    ax[n].text(100,200,filename[i],color='w',fontsize=20)
    ax[n].text(500,200,round(np.mean(image_data[i][100:200,600:800]),2),color='w',fontsize=20)
    divider = make_axes_locatable(ax[n])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(img, cax=cax)
    n = n+1

There is a small difference of signal at the beginning and end of the night.  
The 45230.fits BIAS is useless.

#### Sky FLATS files 

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=3, figsize=(14, 8))
n = 0
for i in range(0,3):
    img = ax[n].imshow(image_data[i], cmap='gray', origin='lower',vmin=600,vmax=1000)
    ax[n].set_xlabel('X axis')
    ax[n].set_ylabel('Y axis')
    ax[n].text(150,200,filename[i],color='w',fontsize=20)
    ax[n].text(500,200,round(np.mean(image_data[i][100:200,600:800]),2),color='w',fontsize=20)
    divider = make_axes_locatable(ax[n])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(img, cax=cax)
    n = n+1

#### Arcs

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=3, figsize=(10, 8))
n = 0
for i in range(28,31):
    img = ax[n].imshow(image_data[i], cmap='gray', origin='lower',vmin=600,vmax=1000)
    ax[n].set_xlabel('X axis')
    ax[n].set_ylabel('Y axis')
    ax[n].text(150,200,filename[i],color='w',fontsize=20)
    ax[n].text(540,200,round(np.mean(image_data[i][100:200,600:800]),2),color='w',fontsize=20)
    divider = make_axes_locatable(ax[n])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(img, cax=cax)
    n = n+1

#### Lamp FLATS files 

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=6, figsize=(16, 14))
n = 0
for i in range(39,45):
    img = ax[n].imshow(image_data[i], cmap='gray', origin='lower',vmin=600,vmax=50000)
    ax[n].set_xlabel('X axis')
    ax[n].set_ylabel('Y axis')
    ax[n].text(150,200,filename[i],color='k',fontsize=20)
    ax[n].text(500,200,round(np.mean(image_data[i][100:200,600:800]),2),color='k',fontsize=20)
    divider = make_axes_locatable(ax[n])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(img, cax=cax)
    n = n+1

#### Science images

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=9, figsize=(16, 24))
n = 0
for i in range(18,27):
    img = ax[n].imshow(image_data[i], cmap='gray', origin='lower',vmin=670,vmax=750)
    ax[n].set_xlabel('X axis')
    ax[n].set_ylabel('Y axis')
    ax[n].text(150,200,filename[i],color='w',fontsize=20)
    ax[n].text(500,200,round(np.mean(image_data[i][100:200,600:800]),2),color='w',fontsize=20)
    divider = make_axes_locatable(ax[n])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(img, cax=cax)
    n = n+1

### Selecting files by type 
We wish to create lists containing the BIAS files to build the master BIAS or those scientific files taken with the same filter to apply the corresponding Flat Field, to name two examples.

In [ ]:
from ccdproc import ImageFileCollection
from ccdproc.utils.sample_directory import sample_directory_with_files
ic_all = ImageFileCollection(directory, keywords='*')
print(ic_all.summary.colnames)

In [ ]:
ic_all.summary['file', 'imagetyp', 'exptime', 'naxis1', 'naxis2']

In [ ]:
keys = ['file','OBJECT' , 'EXPTIME' , 'OBSTYPE' , 'CCDSTEMP', 'CCDATEMP']
ic1 = ImageFileCollection(directory, keywords=keys) # only keep track of keys
ic1.summary.colnames

In [ ]:
# BIAS 
matches = (ic1.summary['OBSTYPE'] == 'BIAS') 
bias_list = ic1.summary['file'][matches]
for i in range(len(bias_list)):
    HDUList_object = fits.open(directory+bias_list[i])
    primary_HDU = HDUList_object[0]
    header = primary_HDU.header
    print('BIAS',i,bias_list[i],header['EXPTIME'],header['CCDSTEMP'],header['CCDATEMP'])

In [ ]:
# DARKS
matches = (ic1.summary['OBSTYPE'] == 'DARK') 
dark_list = ic1.summary['file'][matches]
for i in range(len(dark_list)):
    HDUList_object = fits.open(directory+dark_list[i])
    primary_HDU = HDUList_object[0]
    header = primary_HDU.header
    print('DARK',i,dark_list[i],header['EXPTIME'])

## Easier way to select files

In [ ]:
bias_images = ic1.files_filtered(regex_match=True,imagetyp='bias|light')
print(bias_images)
bias_images

In [ ]:
ic_all.summary[('file', 'imagetyp', 'exptime', 'naxis1', 'naxis2') if 'imagetyp='BIAS']

In [ ]:
bias_images = np.delete(bias_images,2)

In [ ]:
print(bias_images)

In [ ]:
flat_images = ic1.files_filtered(regex_match=True,object='flat')
print(flat_images)

In [ ]:
dark_images = ic1.files_filtered(regex_match=True,object='dark')
print(dark_images)

### Checking dark current

In [ ]:
# BIAS
filename , dark_value, exposure = [], [], []
for file in bias_images:
    HDUList_object = fits.open(directory+file)
    primary_HDU = HDUList_object[0]
    image_data = primary_HDU.data
    header = primary_HDU.header
    filename.append(file)
    dark_value.append(np.mean(image_data))
    exposure.append(header['EXPTIME'])

In [ ]:
# DARKS
for file in dark_images:
    HDUList_object = fits.open(directory+file)
    primary_HDU = HDUList_object[0]
    image_data = primary_HDU.data
    header = primary_HDU.header
    filename.append(file)
    dark_value.append(np.mean(image_data[100:200,600:800]))
    exposure.append(header['EXPTIME'])

for i in range(len(filename)):
    print(filename[i],round(dark_value[i],2),exposure[i])

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(9, 4))
ax.plot(exposure,dark_value,'ro')
ax.set_xlabel('texpo [s]')
ax.set_ylabel('Mean [counts]')

## super BIAS  
(This is just a test. We should remove the overscan first.) 

In [ ]:
bias_images

In [ ]:
image_bias = []
for file in bias_images:
    print(directory+file)
    image = CCDData.read(directory+file, unit="adu")
    image_bias.append(image)

In [ ]:
combiner = Combiner(image_bias)

In [ ]:
master_BIAS = combiner.median_combine()

In [ ]:
# auxiliary function to display a rectangle and compute mean value within it
def draw_rectangle(ax, image_data, x1, x2, y1, y2, color, text=False):
    ax.plot((x1, x1), (y1, y2), color, lw=1)
    ax.plot((x2, x2), (y1, y2), color, lw=1)
    ax.plot((x1, x2), (y1, y1), color, lw=1)
    ax.plot((x1, x2), (y2, y2), color, lw=1)
    if text:
        media = image_data[y1:y2,x1:x2].mean()
        std   = image_data[y1:y2,x1:x2].std()
        ax.text((x1+x2)/2, y1+(y2-y1)/8, str(int(media)), 
                ha='center', va='center', color=color, fontsize=15)        
        ax.text((x1+x2)/2, y2-(y2-y1)/8, str(round(std,1)), 
                ha='center', va='top', color=color, fontsize=15)
    return media, std

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=3, figsize=(14, 9))
img = ax[0].imshow(image_bias[0], cmap='gray', origin='lower',vmin=685,vmax=700)
ax[0].text(100,200,filename[0],color='k',fontsize=20)
draw_rectangle(ax[0], image_bias[0].data , 400, 600, 30, 170, color='w',text=True)
img = ax[1].imshow(image_bias[-1], cmap='gray', origin='lower',vmin=685,vmax=700)
ax[1].text(100,200,filename[-1],color='w',fontsize=20)
draw_rectangle(ax[1], image_bias[-1].data , 400, 600, 30, 170, color='w',text=True)
img = ax[2].imshow(master_BIAS, cmap='gray', origin='lower',vmin=685,vmax=700)
ax[2].text(100,200,'master_BIAS',color='w',fontsize=20)
draw_rectangle(ax[2], master_BIAS.data , 100, 300, 30, 170, color='w',text=True)
draw_rectangle(ax[2], master_BIAS.data , 350, 550, 30, 170, color='w',text=True)
draw_rectangle(ax[2], master_BIAS.data , 600, 800, 30, 170, color='w',text=True)
draw_rectangle(ax[2], master_BIAS.data , 850, 1050, 30, 170, color='w',text=True)